# **Movie Data Analysis**

**Import Packages**

In [210]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

**Pull Rotten Tomatoes Data**

In [211]:
url1 = 'https://www.rottentomatoes.com/franchise/marvel_cinematic_universe'
request1 = requests.get(url1)
bs1 = BeautifulSoup(request1.content)

**Pull Wikipedia Data**

In [212]:
url2 = 'https://en.m.wikipedia.org/wiki/List_of_films_based_on_Marvel_Comics_publications'
tables = pd.read_html(url2)

**Make Rotten Tomatoes Dataframe**

In [ ]:
# Make Dataframe
movie_info = pd.DataFrame()

# Get Titles
titles = bs1.find_all('a', {'data-qa': 'franchise-media-link'})

# Extract the text from the links
title = [movie.get_text(strip=True) for movie in titles]

# Add title to Dataframe
movie_info['title'] = title

In [214]:
# Get Years
years = bs1.find_all('span', {'data-qa': 'franchise-media-year'})

# Extract the years and handle date ranges
def clean_year(year_text):
    # Remove parentheses and strip whitespace
    clean_text = year_text.strip('()').strip()
    
    # If it's a range or contains 'Present', take the first year
    if '-' in clean_text or 'Present' in clean_text:
        return int(clean_text.split('-')[0])
    
    # For standard single years
    return int(clean_text)

# Create the list of years
year = [clean_year(movie.get_text(strip=True)) for movie in years]

# Add year to dataframe
movie_info['year'] = year

In [215]:
# Get Director


# Add director to dataframe
# movie_info['director'] = directors

In [216]:
# Get tomato meter score

# Add critic rating to dataframe
# movie_info['critic_rating'] = tomato_scores

In [217]:
# Get popcorn meter score

# Add user rating to dataframe
# movie_info['user_rating'] = popcorn_scores

**Make Wikipedia Dataframe**

In [220]:
# Get Box office table
box_office = tables[14]

# Drop messy heading using
box_office.columns = box_office.columns.droplevel(0)

# Rename columns
new_names = ['title', 'distributor', 'release_date_us', 'budget', 'rev_opening_weekend_NA', 'rev_NA', 'rev_not_NA', 'rev_worldwide']
box_office.columns = new_names


**Join the Two Tables**

In [221]:
# Join on title column
movie_final = pd.merge(movie_info, box_office, on='title', how='inner')